<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_05/blob/main/test_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Language Model (CLM) test

Check if the fine-tuned **facebook/opt-125m** model got any better on the simple task of Portuguese text generation, after seeing the mc4 pt dataset samples on the `LM_training.ipynb` notebook.

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.2 MB/s eta 0:00:00


In [ ]:
WORKING_FOLDER="drive/MyDrive/unicamp/ia368v_dd/aula_05"

API_KEYS_FILE="/content/drive/MyDrive/unicamp/ia368v_dd/api_keys_20230324.json"

TRAIN_OUTPUT_FOLDER="./trained_model"

NORMALIZED_DATA_BLOCKS_PARTIAL_FILENAME="normalized_samples_block_*"

In [ ]:
import os
from google.colab import drive
import json

In [ ]:
drive.mount('/content/drive', force_remount=True)
os.chdir(WORKING_FOLDER)

Mounted at /content/drive


In [ ]:
# from comet_ml import Experiment

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, default_data_collator, TrainerCallback, pipeline  

from multiprocessing import Pool

import pickle

import torch

from tqdm.auto import tqdm

import glob

import numpy as np

from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
MODEL_NAME="facebook/opt-125m"

In [ ]:
model_2048 = AutoModelForCausalLM.from_pretrained("trained_model_2048/checkpoint-89685-1.2310").to(device)
print('Parameters', model_2048.num_parameters())

Parameters 125239296


In [ ]:
model = AutoModelForCausalLM.from_pretrained("trained_model/checkpoint-50000").to(device)
print('Parameters', model.num_parameters())

Parameters 125239296


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Try model generation using the model method

In [ ]:
input_ids = tokenizer("Era uma vez", return_tensors='pt').input_ids

In [ ]:
output = model.generate(input_ids, 
                        num_beams=10, 
                        num_return_sequences=1, 
                        no_repeat_ngram_size=1, 
                        remove_invalid_values=True, 
                        max_length=200,
                        do_sample=False)

In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Era uma vez um jovem de 18 anos que estava com a família na rua, quando foi abordado por policiais da Delegacia Especializada em Trânsito (DETRAN) no bairro Jardim América. Segundo informações repassadas pela Policia Rodoviária Estadual através do telefone celular o rapaz tentou fugir mas não obteve sucesso e morreu após persegui-lo durante patrulhamento ostensivo para evitar mais detalhes sobre as circunstancialidades acima mencionadas. Ainda conforme relatos constatados nas fotos feitas à época dos assaltantes os mesmos se encontraram sem vida enquanto trabalhav


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Era uma vez  irmão de um homem que morreu a tiros na madrugada desta segunda-feira (28), no bairro Jardim Botânico, zona norte da cidade. De acordo com o jornal "O Globo", dois indivíduos armados invadiram e mataram três adolescentes por volta das 19h30min em frentela à casinha onde estava escondido quando os assaltantes fugiam para tentar roubar dinheiro dos cofres municipais durante as festividades carnavalescas realizadas nos último sábado(29). Eles também agrediu outras menores atingidas nas pernas esquerdoprópoles porem nenhuma foi ferida


In [ ]:
output_2048 = model_2048.generate(input_ids, 
                                  num_beams=10, 
                                  num_return_sequences=1, 
                                  no_repeat_ngram_size=1, 
                                  remove_invalid_values=True, 
                                  max_length=200,
                                  do_sample=False)

In [ ]:
print(tokenizer.decode(output_2048[0], skip_special_tokens=True))

Era uma vez o dia em que eu estava na casa de minha mãe. Quando cheguei ao escritório, não havia esperança alguma para meus filhos: "Você vai ficar comigo agora mesmo", disse-me um pouco antes do início da viagem à cidade onde trabalhei durante quase duas semanas até conhecer outros moradores no bairro Jardim Botânico (que hoje se chama Vila Madalena). Naquela época tudo acontecia bem por causa dos desafios enfrentados entre as prefeituras localizadas nas imediaçõES DO PARQUE NACIONAL DA CIDADE DE SÃO PAUL


### Try model generation using the HF pipeline

In [ ]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
model_original = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

In [ ]:
generator_original = pipeline('text-generation', model=model_original, tokenizer=tokenizer)

Try one phrase completion

In [ ]:
generator("O céu está carregado de ", max_length=40)

[{'generated_text': 'O céu está carregado de ervas, e o sol está escaldante. O céu está escaldante. O céu está'}]

In [ ]:
generator_original("O céu está carregado de", max_length=40)

[{'generated_text': 'O céu está carregado de uma cidade de uma cidade de uma cidade de uma cidade de uma cidade'}]

Try another one

In [ ]:
generator("O pneu do carro está ", max_length=40)

[{'generated_text': 'O pneu do carro está ileso e o motorista não pode dirigir. O problema é que o pneu do carro está '}]

In [ ]:
generator_original("O pneu do carro está ", max_length=40)

[{'generated_text': 'O pneu do carro está ilegalmente.\nO carro está mais importante.'}]

And one more

In [ ]:
generator("O ônibus estava muito ", max_length=40)

[{'generated_text': 'O ônibus estava muito ilegal, mas a Polícia Militar de São Paulo conseguiu prender o motorista que estava com o'}]

In [ ]:
generator_original("O ônibus estava muito ", max_length=40)

[{'generated_text': 'O ônibus estava muito ilegalmente.\nO ônibus estava muito ilegalmente.'}]

Trying the same promtp but adding trailling spaces

In [ ]:
generator("Era uma vez ", max_length=200, do_sample=True)

[{'generated_text': 'Era uma vez irmã do mestre de teologia Sibila Dantas: "Quem está intrigado por alguém, não se desgasta ou desmata. Seguindo-o no caminho do bem, não estará sobrecarregado de qualquer prova material; terá de ser sincero e paciente, enquanto a alguém será deixado para trás; segue por cima pela mesma margem, por várias circunstâncias, ou desce para o nada. Só por Deus. Não perderá os espinhos, nem os tachos da vida." Sibila deve ser uma pessoa diferente e cuidar-se do bem das pessoas que'}]

In [ ]:
generator("Era uma vez ", max_length=200, do_sample=False)

[{'generated_text': 'Era uma vez irmão de um amigo, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu irmão, eu e meu'}]

In [ ]:
generator_original("Era uma vez ", max_length=200, do_sample=False)

[{'generated_text': 'Era uma vez                                                                                                                                                                                                 '}]